Simple testing on videos found on the web + team classification.

In [15]:
!python3 track_football.py --source ../data/demo_cut.mp4 \
--referee-color red --team1-color white --team2-color blue --max-frames 15 \
--classes 0 32 --save-vid --show-vid --save-txt --save-crop

File: /home/andi/FER/year5/PlayerTracking/tracker/yolov5/utils/general.py
ROOT: /home/andi/FER/year5/PlayerTracking/tracker/yolov5
File: /home/andi/FER/year5/PlayerTracking/tracker/yolov5/utils/general.py
ROOT: /home/andi/FER/year5/PlayerTracking/tracker/yolov5
track_football: yolo_weights=/home/andi/FER/year5/PlayerTracking/tracker/weights/yolov5m.pt, appearance_descriptor_weights=/home/andi/FER/year5/PlayerTracking/tracker/weights/osnet_x0_25_msmt17.pt, tracking_method=strongsort, source=../data/demo_cut.mp4, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.5, max_det=1000, device=, show_vid=True, save_txt=True, save_conf=False, save_crop=True, save_vid=True, classes=[0, 32], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/track, name=exp, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, hide_class=False, half=False, dnn=False, eval=False, max_frames=15, referee_color=red, team1_color=white, team2_color=blue
/usr/lib/python3/dist-packa

Now let's try to track on videos from the Croatian league.

In [28]:
!python3 track_football.py --source ../data/t4.mp4 --max-frames 100 --classes 0 32 --show-vid

File: /home/andi/FER/year5/PlayerTracking/tracker/yolov5/utils/general.py
ROOT: /home/andi/FER/year5/PlayerTracking/tracker/yolov5
File: /home/andi/FER/year5/PlayerTracking/tracker/yolov5/utils/general.py
ROOT: /home/andi/FER/year5/PlayerTracking/tracker/yolov5
track_football: yolo_weights=/home/andi/FER/year5/PlayerTracking/tracker/weights/yolov5m.pt, appearance_descriptor_weights=/home/andi/FER/year5/PlayerTracking/tracker/weights/osnet_x0_25_msmt17.pt, tracking_method=strongsort, source=../data/t4.mp4, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.5, max_det=1000, device=, show_vid=True, save_txt=False, save_conf=False, save_crop=False, save_vid=False, classes=[0, 32], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/track, name=exp, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, hide_class=False, half=False, dnn=False, eval=False, max_frames=100, referee_color=None, team1_color=None, team2_color=None
/usr/lib/python3/dist-package